In [60]:
import pickle as pkl
import numpy as np
from matplotlib import pyplot as plt
import pysam
from gtfparse import read_gtf

import pandas as pd

In [73]:
alignement_file_path = {bam_file_path} # string for file alignement file path
alignement_100 = pysam.AlignmentFile({alignement_file_path}) #Import alignement file from STAR alignement tool
multi_mapped_100 = [str(line).split('\t')[0] for line in alignement_100 if int((str(line).split('\t')[11])[8])>1] # Get every reads that can be multimap (NH Flag represent the numbers of loci that the read can be aligned)

#### Get dark_gene and dark_trans

In [75]:
def get_read_per_trans(multimapped): # Sort every multimap reads to their respective transcript
    read_per_trans = {}             # Reads are name by ENG|ENST_(START position in the transcript)
    for read in multimapped:
        trans = read.split('_')[0]
        pos = read.split('_')[1]
        if trans not in read_per_trans:
            read_per_trans.update({trans:[]})
        if trans in read_per_trans:
            read_per_trans[trans].append(int(pos))
    return read_per_trans

In [76]:
read_per_trans_100 = get_read_per_trans(multi_mapped_100)

In [77]:
def get_dark_regions_trans(read_per_trans, trans):  # Regrouped every adjacent reads into on multimap region(dark_region)
    dark_region_end = 0                             # Their is one read starting at every 3 pb
    dark_regions = []
    last_pos = -100
    for pos in sorted(set(read_per_trans[trans])):
        if pos != last_pos + 3:
            if last_pos != -100:
                dark_region_end = last_pos
                dark_region = (dark_region_start, dark_region_end)
                dark_regions.append(dark_region)
            dark_region_start = pos
        last_pos = pos
    dark_region = (dark_region_start, pos)
    dark_regions.append(dark_region)
    return dark_regions

def get_dark_regions(read_per_trans):
    dark_regions_dict = {}
    for trans in read_per_trans.keys():
        dark_regions_list = get_dark_regions_trans(read_per_trans, trans)
        dark_regions_dict.update({trans:dark_regions_list})
    return dark_regions_dict

In [78]:
dark_regions_100 = get_dark_regions(read_per_trans_100)

In [80]:
dark_regions_100_list = []  # Put every dark region into a list
dark_trans = []
for trans in dark_regions_100:
    dark_trans.append(trans)
    for dark_region in dark_regions_100[trans]:
        dark_regions_100_list.append(f'{trans}_{dark_region}')


220673

In [159]:
filtered_dark_regions = [dark_region for dark_region in dark_regions_100_list if dark_region.split('|')[1].split('_')[0] in trans_coords]   # Filter dark regions from 22 chr , X, Y, MT

In [167]:
dark_gene = list(set([trans.split('|')[0] for trans in filtered_dark_regions]))
dark_trans = list(set([trans.split('|')[1].split('_')[0] for trans in filtered_dark_regions]))
print(f'Their is {len(set(dark_gene))} gene with at least 1 multimap site')
print(f'Their is {len(set(dark_trans))} transcript with at least 1 multimap site' )

(24208, 85981)

#### Get total dark region number

In [101]:
def get_real_dr_coords(dark_regions_100_list, trans_list):  # Get the coords of the dark region on the chromosome from de coords that we have which are on the transcript
    real_dr_coords = []
    for dark_region in dark_regions_100_list:
        trans = dark_region.split('|')[1].split('_')[0]
        start = dark_region.split('(')[1].split(',')[0]
        end = dark_region.split(', ')[1].split(')')[0]
        trans_chr, trans_start, trans_end = trans_coords[trans]

In [183]:
dark_region_by_chr = {} # Get every dark region in every chromosome

for dark_region in filtered_dark_regions:
    trans = dark_region.split('|')[1].split('_')[0]
    start = dark_region.split('(')[1].split(',')[0]
    end = dark_region.split(', ')[1].split(')')[0]
    
    trans_chr, trans_start, trans_end = trans_coords[trans]

    dr_start = int(start) + int(trans_start)
    dr_end = int(end) + int(trans_start)

    if trans_chr not in dark_region_by_chr:
        dark_region_by_chr.update({trans_chr:[]})
    dark_region_by_chr[trans_chr].append((dr_start, dr_end))

In [242]:
def remove_overlap_dr(dark_region_by_chr, chr): #Their is overlapping transcript so here we remove the dark regions that could be count more than 1 time
    big_end = 0
    for start, end in dark_region_by_chr[chr]:
        if end>big_end:
            big_end = end
            
    matrix = np.zeros(big_end+10)
    for start, end in dark_region_by_chr[chr]:
        matrix[start:end+1] = 1
    
    
    prev_pos = 0
    chr_dark_region = []
    mat_index = 0
    for pos in matrix:
        if pos == 1 and prev_pos == 0:
            index_start = mat_index
        if pos == 0 and prev_pos == 1:
            index_end = mat_index
            chr_dark_region.append((index_start, index_end))
        prev_pos = pos
        mat_index +=1
    return chr_dark_region

In [243]:
chroms = set([str(x) for x in range(1, 23)] + ['X', 'Y', 'MT']) #Get final dark regions
final_dark_regions_dict_by_chr = {chr:remove_overlap_dr(dark_region_by_chr, chr) for chr in chroms} 

In [250]:
tot_dr = 0      # Count total dark regions from all chromosomes
for chr in final_dark_regions_dict_by_chr:
    tot_dr = len(final_dark_regions_dict_by_chr[chr]) + tot_dr

tot_dr

#### Get biotype %

In [253]:
gtf_file =  read_gtf({gtf_file_path})  # Get every genes and transcripts their biotype from gtf, give the path string for gtf
gtf_file = pd.DataFrame(gtf_file, columns = gtf_file.columns)

chroms = set([str(x) for x in range(1, 23)] + ['X', 'Y', 'MT'])
genes_biotypes_df = gtf_file[((gtf_file['feature']=='gene') & (pd.Series(gtf_file['seqname']).isin(chroms)))==True][['gene_id', 'gene_biotype']].to_dict(orient='records')
trans_biotypes_df = gtf_file[((gtf_file['feature']=='transcript') & (pd.Series(gtf_file['seqname']).isin(chroms)))==True][['transcript_id', 'transcript_biotype']].to_dict(orient='records')


INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version', 'ccds_id']


In [256]:
biotype_groups = {                          # Dictionnary to regroup gtf biotypes annotations into simpler categories
    'processed_transcript':'non_coding',
    'lincRNA':'non_coding',
    'antisense':'non_coding',
    'retained_intron':'non_coding',
    'sense_intronic':'non_coding',
    'sense_overlapping':'non_coding',
    'misc_RNA':'non_coding',
    '3prime_overlapping_ncRNA':'non_coding',
    
    'transcribed_unprocessed_pseudogene':'pseudogene',
    'unprocessed_pseudogene':'pseudogene',
    'processed_pseudogene':'pseudogene',
    'transcribed_processed_pseudogene':'pseudogene',
    'pseudogene':'pseudogene',
    'polymorphic_pseudogene':'pseudogene',
    'transcribed_unitary_pseudogene':'pseudogene',
    'rRNA_pseudogene':'pseudogene',
    'IG_V_pseudogene':'pseudogene',
    'translated_processed_pseudogene':'pseudogene',
    'unitary_pseudogene':'pseudogene',
    'IG_C_pseudogene':'pseudogene',
    'IG_J_pseudogene':'pseudogene',
    'IG_pseudogene':'pseudogene',
    'TR_V_pseudogene':'pseudogene',
    'TR_J_pseudogene':'pseudogene',
    
    'protein_coding':'protein_coding',
    'IG_C_gene':'protein_coding',
    'IG_V_gene':'protein_coding',
    'TR_C_gene':'protein_coding',
    'TR_J_gene':'protein_coding',
    'TR_D_gene':'protein_coding',
    'IG_J_gene':'protein_coding',
    'IG_D_gene':'protein_coding',
    'IG_gene':'protein_coding',
    
    'nonsense_mediated_decay':'non_coding',
    'miRNA':'non_coding',
    'TEC':'non_coding',
    'rRNA':'non_coding',
    'bidirectional_promoter_lncRNA':'non_coding',
    'snRNA':'non_coding',
    'scaRNA':'non_coding',
    'scRNA':'non_coding',
    'sRNA':'non_coding',
    'non_stop_decay':'non_coding',
    'snoRNA':'non_coding',
    'macro_lncRNA':'non_coding',
    'non_coding':'non_coding',
    'vaultRNA':'non_coding',
    
    'TR_V_gene':'other',
    'Mt_rRNA':'other',
    'Mt_tRNA':'other',
    'ribozyme':'other'
}

In [269]:
trans_biotype = {trans['transcript_id']:trans['transcript_biotype'] for trans in trans_biotypes_df} # Make dictionnaries genes/transcript : their biotype
gene_biotype = {gene['gene_id']:gene['gene_biotype'] for gene in genes_biotypes_df}

In [267]:
trans_pseudogenes = [trans for trans in set(dark_trans) if biotype_groups[trans_biotype[trans]] == 'pseudogene']    # % of transcripts having at least 1 multimap that are annotated pseudogenes
pseudo_pourc = len(trans_pseudogenes)/len(set(dark_trans))*100
print(f'{pseudo_pourc}% of which are pseudogenes transcripts')

4.942952512764448% of which are pseudogenes transcripts


In [274]:
trans_protein_coding = [trans for trans in set(dark_trans) if biotype_groups[trans_biotype[trans]] == 'protein_coding']     # % of transcripts having at least 1 multimap that are annotated protein_coding
protein_coding_pourc = len(trans_protein_coding)/len(set(dark_trans))*100
print(f'{protein_coding_pourc}% of which are protein_coding transcripts')

52.158034914690454% of which are protein_coding transcripts


In [275]:
trans_non_coding = [trans for trans in set(dark_trans) if biotype_groups[trans_biotype[trans]] == 'non_coding']     # % of transcripts having at least 1 multimap that are annotated protein_coding
non_coding_pourc = len(trans_non_coding)/len(set(dark_trans))*100
print(f'{non_coding_pourc}% of which are non_coding transcripts')

42.88505600074435% of which are non_coding transcripts


In [273]:
tot_pseudogenes = [gene for gene in gene_biotype if biotype_groups[gene_biotype[gene]] == 'pseudogene']     # % of pseudogenes that have a at least 1 multimap site
dark_pseudogenes = [gene for gene in set(dark_gene) if biotype_groups[gene_biotype[gene]] == 'pseudogene']
pseudo_pourc = len(dark_pseudogenes)/len(set(tot_pseudogenes))*100
print(f'{pseudo_pourc}% of pseudogenes have at least 1 dark regions')

29.136548823761338% of pseudogenes have at least 1 dark regions


In [277]:
gene_pseudogenes = [gene for gene in set(dark_gene) if biotype_groups[gene_biotype[gene]] == 'pseudogene']      # % of gene having at least 1 multimap site that are annotated pseudogenes 
pseudo_pourc = len(gene_pseudogenes)/len(set(dark_gene))*100
print(f'{pseudo_pourc}% of which are pseudogenes genes')

18.316259087904825% of which are pseudogenes genes
